<a href="https://colab.research.google.com/github/StefannyEscobar/Optimization-Algorithms/blob/main/ELECTRE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Librerias

In [ ]:
import pandas as pd
import numpy as np
import random
import math

## Ingresamos la matriz de datos

In [ ]:
alternatives = np.array([[7,15,12,34,9,2,5,8,40,17]
,[5,14,20,26,9,2,4,15,22,16],
[11,19,21,31,1,11,4,42,14,18],
[13,1,24,27,35,6,1,5,4,2]])
#Convertimos los datos de entrada en una matriz
pd_alter = pd.DataFrame(data = alternatives,
                        index = ['A1','A2', 'A3', 'A4'],
                        columns=['c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9', 'c10'])
pd_alter

,c1,c2,c3,c4,c5,c6,c7,c8,c9,c10
A1,7,15,12,34,9,2,5,8,40,17
A2,5,14,20,26,9,2,4,15,22,16
A3,11,19,21,31,1,11,4,42,14,18
A4,13,1,24,27,35,6,1,5,4,2


# <center> Punto 1A

### Implementación ELECTRE
#### <span style="color:Blue"> Paso 1:

$En\ primer\ lugar\ le\ damos\ los\ pesos\ a\ cada\ criterio\ para\ construir\ la\ matriz\ de\ desión$

Para esto se decidio hacerlo de manera aleatoria ya que no se tiene preferencia sobre ninguna

In [ ]:
n = 10
nums = [random.random() for _ in range(n)]
suma = sum(nums)
weigth = [num/suma for num in nums]
w = [0.16080725415783573,
0.20565938230877176,
0.1785406855161036,
0.0321902504032635,
0.0289215951779111,
0.137201741307777,
0.1315644222831187,
0.03016823558028442,
0.06923148936889213,
0.02571494389604221]

#### <span style="color:Blue"> Paso 3:

$Construir\ la\ matriz\ de\ decisión\ $
$donde:$
La norma para $p=2$
$$v_i=\frac{max (r_i)-r_i}{max (r_i)-min (r_i)}$$


## Norma Electre


In [ ]:
# Normalización de la matriz utilizando el método Electre
def normelectre(mat):
    sizes  = np.shape(mat)
    m = sizes[0]
    n = sizes[1]
    norm_matrix = np.zeros((m, n))
    for j in range(n):
        max_value = np.max(mat[:, j])
        min_value = np.min(mat[:, j])
        for i in range(m):
            if max_value == min_value:
                norm_matrix[i, j] = 1
            else:
                norm_matrix[i, j] = (max_value - mat[i, j] ) / (max_value - min_value)
    return norm_matrix
matriz = normelectre(alternatives)
pd_norm_matrix = pd.DataFrame(data=matriz,
                            index=['A1', 'A2', 'A3', 'A4'],
                            columns=['c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9', 'c10'])

pd_norm_matrix


,c1,c2,c3,c4,c5,c6,c7,c8,c9,c10
A1,0.75,0.222222,1.000000,0.000,0.764706,1.000000,0.00,0.918919,0.000000,0.0625
A2,1.00,0.277778,0.333333,1.000,0.764706,1.000000,0.25,0.729730,0.500000,0.1250
A3,0.25,0.000000,0.250000,0.375,1.000000,0.000000,0.25,0.000000,0.722222,0.0000
A4,0.00,1.000000,0.000000,0.875,0.000000,0.555556,1.00,1.000000,1.000000,1.0000


In [ ]:
arr = pd_alter.agg(['min', 'max'])
ar =np.matrix(arr)
ar[1]-ar[0]


matrix([[ 8, 18, 12,  8, 34,  9,  4, 37, 36, 16]])

In [ ]:
# Definir la función de concordancia utilizando el criterio de Pareto

# Construir la matriz de concordancia
def matriz_concordance(norm_matrix,w):
    sizes  = np.shape(norm_matrix)
    m = sizes[0]
    n = sizes[1]
    concordance_matrix = np.zeros((m, m))
    for i in range(m):
        for j in range(m):
            if i == j:
                concordance_matrix[i,j] = 0
            else:
                row_i = norm_matrix[i,:]
                row_j = norm_matrix[j,:]
                concordant_criteria = []
                for k in range(n):
                    if row_i[k]==row_j[k] and i<j:
                        concordant_criteria.append(w[k])
                        val = sum(concordant_criteria) 
                    if row_i[k]>row_j[k]:
                        concordant_criteria.append(w[k])
                        val = sum(concordant_criteria)
                concordance_matrix[i,j] = val
    return concordance_matrix
con = matriz_concordance(matriz,w)
pd_norm_matrix = pd.DataFrame(data=con,
                            index=['A1', 'A2', 'A3', 'A4'],
                            columns=['A1', 'A2', 'A3', 'A4'])

pd_norm_matrix


,A1,A2,A3,A4
A1,0.000000,0.374832,0.738092,0.505471
A2,0.625168,0.000000,0.901847,0.537662
A3,0.261908,0.098153,0.000000,0.368270
A4,0.494529,0.462338,0.631730,0.000000


## Matriz +1/2

$Esta\ matriz\ se\ define\ como:$

$$\sum_{j:C_j(A_i)>C_j(A_k)} w_j+\frac{1}{2}$$

Por lo que el resultado anterior no estaría siguiendo la definición

In [ ]:
from pandas._config.config import config_prefix
# Definir la función de concordancia utilizando el criterio de Pareto

# Construir la matriz de concordancia
def matriz_concordance(norm_matrix,w):
    sizes  = np.shape(norm_matrix)
    m = sizes[0]
    n = sizes[1]
    concordance_matrix = np.zeros((m, m))
    for i in range(m):
        for j in range(m):
            if i == j:
                concordance_matrix[i,j] = 0
            else:
                row_i = norm_matrix[i,:]
                row_j = norm_matrix[j,:]
                concordant_criteria = []
                for k in range(n):
                    if row_i[k]==row_j[k] and i<j:
                        concordant_criteria.append(w[k])
                        val = sum(concordant_criteria) 
                    if row_i[k]>row_j[k]:
                        concordant_criteria.append(w[k])
                        val = sum(concordant_criteria)
                concordance_matrix[i,j] = val+0.5
    return concordance_matrix
conp = matriz_concordance(matriz,w)
pd_norm_matrixp = pd.DataFrame(data=conp,
                            index=['A1', 'A2', 'A3', 'A4'],
                            columns=['A1', 'A2', 'A3', 'A4'])

pd_norm_matrixp


,A1,A2,A3,A4
A1,0.000000,0.874832,1.238092,1.005471
A2,1.125168,0.000000,1.401847,1.037662
A3,0.761908,0.598153,0.000000,0.868270
A4,0.994529,0.962338,1.131730,0.000000


$Construir\ la\ matriz\ de\ decisión\ ponderada $
$donde:$

$$\overline{v}_{ij} =v_{ij}\cdot w_j, \forall j = 1,2,...,n$$

$Donde:$
$$\sum W_j =1$$

In [ ]:
def normalizepesos(vec, mat):
    sizes  = np.shape(mat)
    m = sizes[0]
    n = sizes[1]
    v = np.zeros((m, n))
    for j in range(n):
        for i in range(m):
                v[i][j] = mat[i][j] * vec[j]
    return v
normalizada=matriz
normalizadap = normalizepesos(w,normalizada)
pd_normp = pd.DataFrame(data = normalizadap,
                        index = ['A1','A2', 'A3', 'A4'],
                        columns=['c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9', 'c10'])
pd_normp

,c1,c2,c3,c4,c5,c6,c7,c8,c9,c10
A1,0.120605,0.045702,0.178541,0.000000,0.022117,0.137202,0.000000,0.027722,0.000000,0.001607
A2,0.160807,0.057128,0.059514,0.032190,0.022117,0.137202,0.032891,0.022015,0.034616,0.003214
A3,0.040202,0.000000,0.044635,0.012071,0.028922,0.000000,0.032891,0.000000,0.050001,0.000000
A4,0.000000,0.205659,0.000000,0.028166,0.000000,0.076223,0.131564,0.030168,0.069231,0.025715


## Matriz de discordancia
#### Esta matriz esta definida de la siguiente manera

$$D(i,k)= \frac{max_{(i,k)|\overline{v_j}(i)<\overline{v_j}(k)}\overline{v_j}(k)-\overline{v_j}(i)}{max_{\forall(i,k)}|\overline{v_j}(k)-\overline{v_j}(i)|}$$

In [ ]:
from pandas._config.config import config_prefix
# Construir la matriz de concordancia
def matriz_disconcordance(norm_matrix,w):
    sizes  = np.shape(norm_matrix)
    m = sizes[0]
    n = sizes[1]
    disconcordance_matrix = np.zeros((m, m))
    for i in range(m):
        for k in range(m):
            if i == k:
                disconcordance_matrix[i,k] = 0
            else:
                row_i = norm_matrix[i,:]
                row_k = norm_matrix[k,:]
                disconcordant_criteria = []
                for j in range(n):
                  maxi = max(abs(row_i-row_k))
                  if row_i[j]<row_k[j]:
                    dif = max(row_i-row_k)
                    disconcordance_matrix[i,k] = dif/maxi
    return disconcordance_matrix
dis = matriz_disconcordance(normalizadap,w)
pd_norm_matrixp = pd.DataFrame(data=dis,
                            index=['A1', 'A2', 'A3', 'A4'],
                            columns=['A1', 'A2', 'A3', 'A4'])

pd_norm_matrixp


,A1,A2,A3,A4
A1,0.000000,1.000000,1.0,1.000000
A2,0.337753,0.000000,1.0,1.000000
A3,0.364431,0.112133,0.0,0.217034
A4,0.895915,0.923663,1.0,0.000000


# Definión de umbrales

$$aSb ⟺ C(a,b) \geq c^*\ y\ D(a,b) \leq d^*$$



In [ ]:
elementos = 0
sumatoria = 0
for fila in dis:
    for elemento in fila:
        sumatoria += elemento
        elementos += 1

D = sumatoria / elementos
D

0.553183094242232

In [ ]:
elementos = 0
sumatoria = 0
for fila in con:
    for elemento in fila:
        sumatoria += elemento
        elementos += 1

c = sumatoria / elementos
c

0.375

In [ ]:
def domincon(c, mat):
    sizes  = np.shape(mat)
    m = sizes[0]
    n = sizes[1]
    v = np.zeros((m, n))
    for j in range(n):
      for i in range(m):
        if mat[i][j] <=c:
          v[i][j] = 0 
        else:
          v[i][j] = 1
    return v   
m_condordanciaDom = domincon(c,con)
condordanciaDom = pd.DataFrame(data = m_condordanciaDom,
                        index = ['A1','A2', 'A3', 'A4'],
                        columns= ['A1','A2', 'A3', 'A4'])
condordanciaDom

,A1,A2,A3,A4
A1,0.0,0.0,1.0,1.0
A2,1.0,0.0,1.0,1.0
A3,0.0,0.0,0.0,0.0
A4,1.0,1.0,1.0,0.0


In [ ]:
def domindis(d, mat):
    sizes  = np.shape(mat)
    m = sizes[0]
    n = sizes[1]
    v = np.zeros((m, n))
    for j in range(n):
      for i in range(m):
        if mat[i][j] >=d:
            v[i][j] = 1 
        else:
          v[i][j] = 0
    return v   
m_discondordanciaDom = domindis(D,dis)
discondordanciaDom = pd.DataFrame(data = m_discondordanciaDom,
                        index = ['A1','A2', 'A3', 'A4'],
                        columns= ['A1','A2', 'A3', 'A4'])
discondordanciaDom

,A1,A2,A3,A4
A1,0.0,1.0,1.0,1.0
A2,0.0,0.0,1.0,1.0
A3,0.0,0.0,0.0,0.0
A4,1.0,1.0,1.0,0.0


In [ ]:
def domin_agg(dis, con):
    sizes  = np.shape(dis)
    m = sizes[0]
    n = sizes[1]
    v = np.zeros((m, n))
    for j in range(n):
      for i in range(m):
        v[i][j] = dis[i][j]*con[i][j]
    return v   
domin_agg = domin_agg(m_discondordanciaDom,m_condordanciaDom)
domin_agg_m = pd.DataFrame(data = domin_agg,
                        index = ['A1','A2', 'A3', 'A4'],
                        columns= ['A1','A2', 'A3', 'A4'])
domin_agg_m

,A1,A2,A3,A4
A1,0.0,0.0,1.0,1.0
A2,0.0,0.0,1.0,1.0
A3,0.0,0.0,0.0,0.0
A4,1.0,1.0,1.0,0.0
